In [ ]:
# # 코랩에서 한글 폰트를 사용하기 위한 과정

# # 1. 아래코드 실행하여 폰트 다운
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

# # 2. 코랩의 런타임 다시시작(단축키:ctrl+M)
# # 3. 런타임 다시시작 완료했으면, 아래코드 실행

## 1. 라이브러리 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


---
* 필요한 라이브러리 불러오기

In [ ]:
pip install plotly

In [ ]:
pip install prophet

In [ ]:
from prophet.plot import plot_plotly, plot_components_plotly
from fbprophet.plot import add_changepoints_to_plot
from sklearn.metrics import mean_squared_error, r2_score
from fbprophet import Prophet
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import datetime
from math import sqrt
import missingno as msno
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
plt.style.use('seaborn')
sns.set(font='NanumBarunGothic', font_scale=2.5)
# sns.set(font='Malgun Gothic', font_scale=2.5) 
plt.rcParams['axes.unicode_minus'] = False

# plotly 범례이름 설정 함수
def custom_legend_name(new_names):
    for i, new_name in enumerate(new_names):
        fig.data[i].name = new_name

## 2. 시계열 데이터 전처리

In [ ]:
final=pd.read_csv('/content/drive/Shareddrives/Junnesse 조(빅데이터 분석 경진대회(2차))/데이터셋/final2.csv')
print(final.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66748 entries, 0 to 66747
Data columns (total 35 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   시도명       66748 non-null  object 
 1   시군구명      66748 non-null  object 
 2   개방자치단체코드  66748 non-null  int64  
 3   개방서비스명    66748 non-null  object 
 4   인허가일자     66748 non-null  object 
 5   영업상태구분코드  66748 non-null  int64  
 6   사업장명      66748 non-null  object 
 7   면적        66748 non-null  float64
 8   폐휴업일자     27646 non-null  object 
 9   좌표결측      66748 non-null  float64
 10  경도        66748 non-null  float64
 11  위도        66748 non-null  float64
 12  지역        66748 non-null  object 
 13  호텔        66748 non-null  int64  
 14  모텔        66748 non-null  int64  
 15  하우스       66748 non-null  int64  
 16  호스텔       66748 non-null  int64  
 17  스테이       66748 non-null  int64  
 18  텔         66748 non-null  int64  
 19  리조트       66748 non-null  int64  
 20  콘도        66748 non-null  in

In [ ]:
# 월별 창업수로 데이터 생성
final.loc[:,'인허가년월'] = final.인허가일자.map(lambda x:x[:7]).astype('datetime64[ns]')
df = final.pivot_table(index = '인허가년월',
                       values = '면적', aggfunc='count').rename(columns={'면적':'창업수'})

In [ ]:
# 1960년부터 2021년까지의 창업매장 수 데이터 결측치 채우기
df=df.reset_index()
df.인허가년월 = df.인허가년월.astype('datetime64[ns]')
pr_m = pd.period_range(start = '1960-01-01',
                   	   end = '2021-12-01',
                       freq = 'M')           
all = pd.DataFrame(data=None, index=pr_m).reset_index().rename(columns = {'index':'인허가년월'})
all.인허가년월 = all.인허가년월.astype('str').astype('datetime64[ns]')
op = pd.merge(all,df.reset_index(),on='인허가년월',how='outer')[['인허가년월', '창업수']].fillna(0)
print(op.info())

In [ ]:
# 월별 폐휴업수로 데이터 생성
final.loc[final.폐휴업일자.notnull(),'폐휴업년월'] = final.loc[final.폐휴업일자.notnull(),
                                                     '폐휴업일자'].map(lambda x:x[:7]).astype('datetime64[ns]')
df = final.pivot_table(index = '폐휴업년월', values = '면적',
                       aggfunc='count').rename(columns={'면적':'폐휴업수'})

In [ ]:
# 1960년부터 2021년까지 월별 폐휴업매장의 수 데이터 결측치 채우기
df=df.reset_index()
df.폐휴업년월 = df.폐휴업년월.astype('datetime64[ns]')
pr_m = pd.period_range(start = '1960-01-01',
                   	   end = '2021-12-01',
                       freq = 'M')           
all = pd.DataFrame(data=None, index=pr_m).reset_index().rename(columns = {'index':'폐휴업년월'})
all.폐휴업년월 = all.폐휴업년월.astype('str').astype('datetime64[ns]')
cl = pd.merge(all,df.reset_index(),on='폐휴업년월',how='outer')[['폐휴업년월', '폐휴업수']].fillna(0)
print(cl.info())

In [ ]:
# 월별 날짜를 기준으로 창업수와 폐휴업수 통합하여 생성
op = op.rename(columns = {'인허가년월':'날짜'})
cl = cl.rename(columns = {'폐휴업년월':'날짜'})
day = pd.merge(op,cl,on='날짜',how='inner')

# 1960년 이후의 데이터만 인덱싱
dy = day[day.날짜>=datetime.datetime(1960, 1, 1)]

dy.head()

,날짜,창업수,폐휴업수
0,1960-01-01,2.0,0.0
1,1960-02-01,0.0,0.0
2,1960-03-01,3.0,0.0
3,1960-04-01,2.0,0.0
4,1960-05-01,0.0,0.0


In [ ]:
# 2020년 이전은 past로, 이후는 future로 나눔
day = dy.rename(columns={'날짜':'day'})
day.set_index('day', inplace=True)
past = day[day.index<datetime.datetime(2020, 1, 1)] # 1960-01-01 ~ 2019-12-01
future = day[day.index>=datetime.datetime(2020, 1, 1)] # 2020-01-01 ~ 2021-12-01

past_op =  past[['창업수']] # 창업에 대한 2020년 전 데이터
past_cl =  past[['폐휴업수']] # 폐업에 대한 2020년 후 데이터
future_op =  future[['창업수']] # 창업에 대한 2020년 전 데이터
future_cl =  future[['폐휴업수']] # 폐업에 대한 2020년 후 데이터

-----

## 3. < **월별** **창업매장수** **데이터** >

### [시계열 데이터 살펴보기]

In [ ]:
fig = px.line(day.창업수)
fig.update_xaxes(rangeslider_visible=True)
fig.update_layout(title='<b>월별 창업수 추이</b>', yaxis=dict({'title':'창업수'}))
fig.show()

### [Facebook Prophet 활용하여 시계열 예측]


###### Prophet 모델 학습

In [ ]:
# prophet을 사용하기 위해서는 다음과 같이 피처의 이름을 변경해야 합니다 : 'ds', 'y'
past_train_df = past_op.reset_index()
past_df = past_train_df.rename(columns={'day':'ds','창업수':'y'})

# 월별 280개 이상의 창업수 데이터는 이상치라고 판단
# 정확도를 높이기 위함
past_df.loc[past_df['y'] > 280, 'y'] = None

# prophet 모델을 학습합니다.
prophet = Prophet(seasonality_mode='multiplicative',
                  yearly_seasonality=True,
                  weekly_seasonality=False, daily_seasonality=False,
                  changepoint_prior_scale=1.0)
prophet.fit(past_df)


# 25개월단위의 미래를 예측합니다.(2019년 12월 31일자도 포함되어 25개월을 예측)
future_data = prophet.make_future_dataframe(periods=25, freq='M')
forecast_data = prophet.predict(future_data)

# 예측한 데이터의 날짜, 예측값, 최소예측값, 최대예측값 컬럼들만 추출
forecast_data[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head()

,ds,yhat,yhat_lower,yhat_upper
0,1960-01-01,0.519979,-33.345571,31.824587
1,1960-02-01,0.551489,-33.754884,35.189527
2,1960-03-01,0.696597,-34.259567,34.716200
3,1960-04-01,0.905624,-33.613405,33.626088
4,1960-05-01,1.083733,-30.873445,33.861567


-----

##### 2020년 이전 창업수 데이터를 기반으로 학습한 예측 결과를 시각화합니다.
- 검은 점: 실제 데이터
- 파란 실선: 예측한 값
- 하늘색 폭: 최솟값과 최댓값 사이 
- 빨간 실선: 트렌드
- 빨간 점선: 국내 첫 코로나 확진 날짜 

In [ ]:
fig = plot_plotly(prophet, forecast_data, changepoints=True, trend=True)
fig.add_vline(x='2020-01-20', line=dict(color='red'), line_dash='dot')
fig.show()

In [ ]:
# seasonality_mode로 설정했었던 기간별 트렌드 정보를 시각화합니다.
fig2 = plot_components_plotly(prophet, forecast_data)
fig2.show()

-----

#####모델의 성능
- 실제 창업매장 수와 예측한 가격간의 차이 : 첫 25개월과 마지막 25개월은 제외하고 계산합니다.

In [ ]:
y = past_df.fillna(0) # 실제 데이터의 결측치 처리
y_pred = forecast_data.fillna(0) # 예측된 데이터의 결측치 처리
y = y.y.values[25:] # 첫 25개월을 제외한 실제 데이터
y_pred = y_pred.yhat.values[25:-25] # 첫 25개월, 마지막 25개월을 제외한 예측 데이터

In [ ]:
rmse = sqrt(mean_squared_error(y, y_pred))
r2 = r2_score(y, y_pred)
print(rmse)
print(r2)

34.60808228348959
0.6341460484773616


-----

##### 실제 데이터와의 비교
- 예측 결과인 마지막 24개월의 예측값을 실제 데이터와 비교해봅니다.
<br> 2020-01-01 ~ 2021-12-01

In [ ]:
# 테스트 데이터셋을 불러옵니다.
future_test_df = future_op.reset_index()
future_test_df = future_test_df.rename(columns={'day':'ds','창업수':'y'})

pred_y = forecast_data[-24:] # 마지막 24개월의 예측 데이터입니다. (2020년 1월 ~ 2021년 12월)
test_y = future_test_df # 실제 데이터입니다. (2020년 1월 ~ 2021년 12월)

In [ ]:
fig = px.line(pred_y, x='ds', y='yhat', markers=True) # 모델이 예상한 창업매장수 그래프입니다. (빨간 점선)
fig1= px.line(pred_y, x='ds', y='yhat_lower') # 모델이 예상한 최소 창업매장 수 그래프입니다. (노란 점선)
fig2= px.line(pred_y, x='ds', y='yhat_upper') # 모델이 예상한 최대 창업매장 수 그래프입니다 (회색 점선)
fig3= px.line(test_y, x='ds', y='y') # 실제 창업매장 수 그래프입니다. (파란 실선)

# 그래프 세부설정 및 합치기
fig.update_traces(line=dict(color='red'), line_dash='dot')
fig1.update_traces(line=dict(color='gold'), line_dash='dash')
fig2.update_traces(line=dict(color='gray'), line_dash='dash')
(fig.add_trace(fig1.data[0]), fig.add_trace(fig2.data[0]), fig.add_trace(fig3.data[0]))
fig.update_layout(xaxis=dict({'title':'After Covid-19'}), yaxis=dict({'title':'OPEN'}))
custom_legend_name(['pred_y','pred_y_lower','pred_y_upper','tset_y'])
fig.show()

In [ ]:
fig = px.line(pred_y, x='ds', y='yhat', markers=True) # 모델이 예상한 창업매장 수 그래프입니다.
fig1 = px.line(test_y, x='ds', y='y') # 실제 창업매장 수 그래프입니다.

# 그래프 세부설정 및 합치기
fig.update_traces(line=dict(color='red'), line_dash='dot')
fig.add_trace(fig1.data[0])
fig.update_layout(xaxis=dict({'title':'After Covid-19'}), yaxis=dict({'title':'OPEN'}))
custom_legend_name(['pred_y','tset_y'])
fig.show()

-----

## 4. < **월별** **폐휴업매장수** **데이터** >

### [시계열 데이터 살펴보기]

In [ ]:
fig = px.line(day.폐휴업수)
fig.update_xaxes(rangeslider_visible=True)
fig.update_layout(title='<b>월별 폐휴업수 추이</b>', yaxis=dict({'title':'폐휴업수'}))
fig.show()

### [Facebook Prophet 활용하여 시계열 예측]

###### Prophet 모델 학습

In [ ]:
# prophet을 사용하기 위해서는 다음과 같이 피처의 이름을 변경해야 합니다 : 'ds', 'y'
past_train_df = past_cl.reset_index()
past_df = past_train_df.rename(columns={'day':'ds','폐휴업수':'y'})

# 월별 280개 이상의 창업수 데이터는 이상치라고 판단
# 정확도를 높이기 위함
# past_df.loc[past_df['y'] > 280, 'y'] = None

# prophet 모델을 학습합니다.
prophet = Prophet(seasonality_mode='multiplicative',
                  yearly_seasonality=True,
                  weekly_seasonality=False, daily_seasonality=False,
                  changepoint_prior_scale=1.0)
prophet.fit(past_df)


# 25개월단위의 미래를 예측합니다.(2019년 12월 31일자도 포함되어 25개월을 예측)
future_data = prophet.make_future_dataframe(periods=25, freq='M')
forecast_data = prophet.predict(future_data)

# 예측한 데이터의 날짜, 예측값, 최소예측값, 최대예측값 컬럼들만 추출
forecast_data[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head()

,ds,yhat,yhat_lower,yhat_upper
0,1960-01-01,-0.045639,-37.598051,40.314277
1,1960-02-01,-0.057379,-41.398067,38.600866
2,1960-03-01,-0.046703,-38.274630,39.307728
3,1960-04-01,-0.042791,-40.263136,37.054413
4,1960-05-01,-0.042339,-39.308200,38.513356


-----

##### 2020년 이전 창업수 데이터를 기반으로 학습한 예측 결과를 시각화합니다.
* 검은 점: 실제 데이터
* 파란 실선: 예측한 값
* 하늘색 폭: 최솟값과 최댓값 사이
* 빨간 실선: 트렌드
* 빨간 점선: 국내 첫 코로나 확진 날짜

In [ ]:
fig = plot_plotly(prophet, forecast_data, changepoints=True, trend=True)
fig.add_vline(x='2020-01-20', line=dict(color='red'), line_dash='dot')
fig.show()

In [ ]:
fig2 = plot_components_plotly(prophet, forecast_data)
fig2.show()

-----

##### 모델의 성능
- 실제 창업매장 수와 예측한 창업매장 수 간의 차이 : 첫 25개월과 마지막 25개월은 제외하고 계산합니다.

In [ ]:
y = past_df.y.values[25:] # 첫 25개월을 제외한 실제 데이터입니다.
y_pred = forecast_data.yhat.values[25:-25] # 첫 25개월, 마지막 25개월을 제외한 예측 데이터입니다.

In [ ]:
# from sklearn.metrics import mean_squared_error, r2_score
# from math import sqrt
rmse = sqrt(mean_squared_error(y, y_pred))
r2 = r2_score(y, y_pred)
print(rmse)
print(r2)

30.98129604720557
0.695093601690137


-----

##### 실제 데이터와의 비교
- 예측 결과인 마지막 24개월의 예측값을 실제 데이터와 비교해봅니다.
<br> 2020-01-01 ~ 2021-12-01

In [ ]:
# 테스트 데이터셋을 불러옵니다.
future_test_df = future_cl.reset_index()
future_test_df = future_test_df.rename(columns={'day':'ds','폐휴업수':'y'})

pred_y = forecast_data[-24:] # 마지막 24개월의 예측 데이터입니다. (2020년 1월 ~ 2021년 12월)
test_y = future_test_df # 실제 데이터입니다. (2020년 1월 ~ 2021년 12월)

In [ ]:
fig = px.line(pred_y, x='ds', y='yhat', markers=True) # 모델이 예상한 폐휴업매장수 그래프입니다. (빨간 점선)
fig1= px.line(pred_y, x='ds', y='yhat_lower') # 모델이 예상한 최소 폐휴업매장 수 그래프입니다. (노란 점선)
fig2= px.line(pred_y, x='ds', y='yhat_upper') # 모델이 예상한 최대 폐휴업매장 수 그래프입니다 (회색 점선)
fig3= px.line(test_y, x='ds', y='y') # 실제 폐휴업매장 수 그래프입니다. (파란 실선)

# 그래프 세부 설정 및 합치기
fig.update_traces(line=dict(color='red'), line_dash='dot')
fig1.update_traces(line=dict(color='gold'), line_dash='dash')
fig2.update_traces(line=dict(color='gray'), line_dash='dash')
(fig.add_trace(fig1.data[0]), fig.add_trace(fig2.data[0]), fig.add_trace(fig3.data[0]))
fig.update_layout(xaxis=dict({'title':'After Covid-19'}), yaxis=dict({'title':'CLOSE'}))
custom_legend_name(['pred_y','pred_y_lower','pred_y_upper','tset_y'])
fig.show()

In [ ]:
fig = px.line(pred_y, x='ds', y='yhat', markers=True) # 모델이 예상한 폐휴업매장 수 그래프입니다.
fig1 = px.line(test_y, x='ds', y='y') # 실제 폐휴업매장 수 그래프입니다.

# 그래프 세부 설정 및 합치기
fig.update_traces(line=dict(color='red'), line_dash='dot')
fig.add_trace(fig1.data[0])
fig.update_layout(xaxis=dict({'title':'After Covid-19'}), yaxis=dict({'title':'CLOSE'}))
custom_legend_name(['pred_y','tset_y'])
fig.show()

-----